In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pancreatic_Cancer"
cohort = "GSE222788"

# Input paths
in_trait_dir = "../../input/GEO/Pancreatic_Cancer"
in_cohort_dir = "../../input/GEO/Pancreatic_Cancer/GSE222788"

# Output paths
out_data_file = "../../output/preprocess/Pancreatic_Cancer/GSE222788.csv"
out_gene_data_file = "../../output/preprocess/Pancreatic_Cancer/gene_data/GSE222788.csv"
out_clinical_data_file = "../../output/preprocess/Pancreatic_Cancer/clinical_data/GSE222788.csv"
json_path = "../../output/preprocess/Pancreatic_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Addition of losartan to FOLFIRINOX and chemoradiation reduces immunosuppression-associated genes, Tregs and FOXP3+ cancer cells in locally advanced pancreatic cancer"
!Series_summary	"Purpose: Adding losartan (LOS) to FOLFIRINOX (FFX) chemotherapy followed by chemoradiation (CRT) resulted in 61% R0 surgical resection in our phase II trial in patients with locally advanced pancreatic cancer (LAPC). Here we identify potential mechanisms of benefit by assessing the effects of neoadjuvant losartan on the stromal tumor microenvironment.  Experimental Design: We performed a gene expression and immunofluorescence (IF) analysis using archived surgical samples from patients treated with LOS+FFX+CRT (NCT01821729), FFX+CRT (NCT01591733) or surgery upfront, without any neoadjuvant therapy. We also conducted a longitudinal analysis of multiple biomarkers in the plasma of treated patients.   Results: In comparison to FFX+CRT, LOS+FFX+CRT downregulated immunosup

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np

# Let's analyze the dataset details and extract needed information

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data using
# nCounter PanCancer Immune Profiling Panel, so it's likely gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary:
# The dict shows {0: ['treatment group: FFX_losartan', 'treatment group: FFX', 'treatment group: Untreated']}

# 2.1 Data Availability
# - Trait (Pancreatic Cancer): Can be derived from the treatment group row (0)
trait_row = 0  # The treatment group row, which can be used to infer trait status
# - Age: Not explicitly mentioned in the characteristics
age_row = None  # Age data is not available
# - Gender: Not explicitly mentioned in the characteristics
gender_row = None  # Gender data is not available

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert treatment group to binary trait indicator (cancer vs no cancer)"""
    if pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # All samples appear to be from pancreatic cancer patients with different treatment groups
    # Since this is a study of LAPC (locally advanced pancreatic cancer) treatments, 
    # all samples likely have pancreatic cancer
    return 1  # All are cancer patients

def convert_age(value):
    """Convert age to continuous value"""
    # This function isn't used since age_row is None
    return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    # This function isn't used since gender_row is None
    return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
# Initial filtering on the usability of the dataset
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=(trait_row is not None)
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Assuming we have the clinical_data DataFrame from a previous step
    # If not available, we need to load it
    try:
        # Try to access clinical_data which should have been defined in a previous step
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical data:")
        print(preview)
        
        # Save the clinical data to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=True)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except NameError:
        print("Error: clinical_data DataFrame not found. It should be defined in a previous step.")


Preview of selected clinical data:
{'GSM6932288': [1.0], 'GSM6932289': [1.0], 'GSM6932290': [1.0], 'GSM6932291': [1.0], 'GSM6932292': [1.0], 'GSM6932293': [1.0], 'GSM6932294': [1.0], 'GSM6932295': [1.0], 'GSM6932296': [1.0], 'GSM6932297': [1.0], 'GSM6932298': [1.0], 'GSM6932299': [1.0], 'GSM6932300': [1.0], 'GSM6932301': [1.0], 'GSM6932302': [1.0], 'GSM6932303': [1.0], 'GSM6932304': [1.0], 'GSM6932305': [1.0], 'GSM6932306': [1.0], 'GSM6932307': [1.0], 'GSM6932308': [1.0], 'GSM6932309': [1.0], 'GSM6932310': [1.0], 'GSM6932311': [1.0], 'GSM6932312': [1.0], 'GSM6932313': [1.0], 'GSM6932314': [1.0], 'GSM6932315': [1.0], 'GSM6932316': [1.0], 'GSM6932317': [1.0], 'GSM6932318': [1.0], 'GSM6932319': [1.0], 'GSM6932320': [1.0], 'GSM6932321': [1.0], 'GSM6932322': [1.0], 'GSM6932323': [1.0], 'GSM6932324': [1.0], 'GSM6932325': [1.0], 'GSM6932326': [1.0], 'GSM6932327': [1.0], 'GSM6932328': [1.0]}
Clinical data saved to ../../output/preprocess/Pancreatic_Cancer/clinical_data/GSE222788.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['A2M-mRNA', 'ABCB1-mRNA', 'ABL1-mRNA', 'ADA-mRNA', 'ADORA2A-mRNA',
       'AICDA-mRNA', 'AIRE-mRNA', 'AKT3-mRNA', 'ALCAM-mRNA', 'AMBP-mRNA',
       'AMICA1-mRNA', 'ANP32B-mRNA', 'ANXA1-mRNA', 'APOE-mRNA', 'APP-mRNA',
       'ARG1-mRNA', 'ARG2-mRNA', 'ATF1-mRNA', 'ATF2-mRNA', 'ATG10-mRNA'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers
# These appear to be human gene symbols with "-mRNA" appended to them
# A2M, ABCB1, ABL1, etc. are standard human gene symbols
# The "-mRNA" suffix indicates these are measuring mRNA expression levels of these genes
# No mapping to gene symbols is required as these are already gene symbols with a suffix

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Since this is a treatment comparison study, redefine trait extraction
def convert_treatment_group(value):
    """Convert treatment group to categorical values"""
    if pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Map treatment groups to numeric values
    if 'FFX_losartan' in value:
        return 0  # FFX with losartan
    elif 'FFX' in value:
        return 1  # FFX without losartan
    elif 'Untreated' in value:
        return 2  # No treatment
    else:
        return None

# Extract clinical features with treatment group as the trait
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait="Treatment_Group",  # Using a different trait name
    trait_row=0,  # The treatment group row
    convert_trait=convert_treatment_group,
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)
print("Extracted clinical features with treatment group as trait")

# Save the clinical features
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features saved to {out_clinical_data_file}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, "Treatment_Group")
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, "Treatment_Group")

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="This dataset contains only pancreatic cancer patients with different treatment groups (FFX_losartan, FFX, Untreated). The trait is treatment group rather than cancer status."
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data quality check failed. Linked data not saved.")

Normalized gene data saved to ../../output/preprocess/Pancreatic_Cancer/gene_data/GSE222788.csv
Extracted clinical features with treatment group as trait
Clinical features saved to ../../output/preprocess/Pancreatic_Cancer/clinical_data/GSE222788.csv
Shape of linked data: (41, 1)
Shape after handling missing values: (0, 1)
Quartiles for 'Treatment_Group':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Treatment_Group' in this dataset is fine.

Abnormality detected in the cohort: GSE222788. Preprocessing failed.
Data quality check failed. Linked data not saved.
